### Regridding
Tools for regridding GPM data 
from native satellite grid to model grid.

In [80]:
import os
from glob import glob
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import xesmf as xe

%run  ../diurnal_research/diurnal_config.py
%run  ../diurnal_research/fetch_model_helper.py

In [79]:
# get all available model names
rel_cmip6_path= '/export/data1/cchristo/CMIP6_precip/pr_3hr_historical/'
rel_cmip5_path = '/export/data1/cchristo/CMIP5_precip/pr_3hr_historical/'
unique_cmip6_models = get_unique_models(rel_cmip6_path)
unique_cmip5_models = get_unique_models(rel_cmip5_path)

In [36]:
regrid_weights_dir = '/export/data1/cchristo/regridding_weights/'

In [38]:
rel_data_dir = '/export/data1/cchristo/'

input_data_dir_cm5 = rel_data_dir  + 'diurnal_analysis_results/CMIP5/' #GFDL-ESM2M/'
input_data_dir_cm6  = rel_data_dir + 'diurnal_analysis_results/CMIP6/' #GFDL-ESM4/'
input_data_dir_gpm = rel_data_dir  + 'diurnal_analysis_results/GPM/'
land_sea_mask_dir = rel_data_dir  + 'land_sea_masks/'
regrid_weights_dir = rel_data_dir + 'regridding_weights/'

In [42]:
ds_gpm = xr.open_dataset(input_data_dir_gpm +  '2000-06_2010-06_precip.nc')
df_gpm = ds_gpm.to_dataframe()

ds_land_sea = xr.open_dataset(land_sea_mask_dir + 'GLDASp4_landmask_1d.nc4')

ds_in = ds_land_sea.copy()
ds_out = ds_gpm.copy()

In [41]:
ds_in

<xarray.Dataset>
Dimensions:     (bnds: 2, lat: 150, lon: 360, time: 1)
Coordinates:
  * lon         (lon) float32 -179.5 -178.5 -177.5 -176.5 ... 177.5 178.5 179.5
  * lat         (lat) float32 -59.5 -58.5 -57.5 -56.5 ... 86.5 87.5 88.5 89.5
  * time        (time) datetime64[ns] 2000-01-01
Dimensions without coordinates: bnds
Data variables:
    time_bnds   (time, bnds) datetime64[ns] ...
    GLDAS_mask  (time, lat, lon) float32 ...
Attributes:
    missing_value:          -9999.0
    time_definition:        constant
    title:                  GLDAS p4 data
    institution:            NASA GSFC
    history:                created on date: Thu Mar 14 10:17:57 2019
    references:             Rodell_etal_BAMS_2004, Kumar_etal_EMS_2006,Peters...
    conventions:            CF-1.6
    website:                https://ldas.gsfc.nasa.gov/gldas/, https://lis.gs...
    MAP_PROJECTION:         EQUIDISTANT CYLINDRICAL
    SOUTH_WEST_CORNER_LAT:  -59.5
    SOUTH_WEST_CORNER_LON:  -179.5
    DX:                     1.0
    DY:                     1.0

In [19]:
# ds_in.transpose('time','lat','lon', 'bnds')

In [44]:
regridder = xe.Regridder(ds_in, ds_out, 
                         method = 'nearest_s2d', 
                         reuse_weights = False, 
                         filename = regrid_weights_dir + 'GLDAS_ls_mask_to_gfdl_cm4.nc')
ds_out = regridder(ds_in['GLDAS_mask'].transpose('time','lat','lon'))

Overwrite existing file: /export/data1/cchristo/regridding_weights/GLDAS_ls_mask_to_gfdl_cm4.nc 
 You can set reuse_weights=True to save computing time.


In [47]:
# ds_out.isel(time = 0).plot.imshow()
ds_out.to_dataset().to_netcdf(land_sea_mask_dir + 'GLDAS_rg_gfdl_cm4.nc')

In [72]:
for model_ii in list(cmip6_to_cmip5_map.keys()):
    print(model_ii)
    ds_in = ds_land_sea.copy().isel(time = 0)
    ds_out = xr.open_dataset(input_data_dir_cm6 + model_ii + '/' + '1985-01_2006-01_precip.nc')
    
    regridder = xe.Regridder(ds_in, ds_out, 
                         method = 'nearest_s2d', 
                         reuse_weights = False, 
                         filename = regrid_weights_dir + 'GLDAS_ls_mask_to_' + model_ii + '.nc')
    
    ds_rgg = regridder(ds_in['GLDAS_mask'].transpose('lat','lon'))
    ds_out['land_sea_mask'] = ds_rgg
    ds_out.to_netcdf(input_data_dir_cm6 + model_ii + '/' + '1985-01_2006-01_precip_ls_mask.nc')

GFDL-ESM4
Overwrite existing file: /export/data1/cchristo/regridding_weights/GLDAS_ls_mask_to_GFDL-ESM4.nc 
 You can set reuse_weights=True to save computing time.
GFDL-CM4
Create weight file: /export/data1/cchristo/regridding_weights/GLDAS_ls_mask_to_GFDL-CM4.nc
FGOALS-g3
Create weight file: /export/data1/cchristo/regridding_weights/GLDAS_ls_mask_to_FGOALS-g3.nc
IPSL-CM6A-LR
Create weight file: /export/data1/cchristo/regridding_weights/GLDAS_ls_mask_to_IPSL-CM6A-LR.nc
MIROC6
Create weight file: /export/data1/cchristo/regridding_weights/GLDAS_ls_mask_to_MIROC6.nc
MRI-ESM2-0
Create weight file: /export/data1/cchristo/regridding_weights/GLDAS_ls_mask_to_MRI-ESM2-0.nc


In [73]:
for model_ii in list(cmip6_to_cmip5_map.values()):
    print(model_ii)
    ds_in = ds_land_sea.copy().isel(time = 0)
    ds_out = xr.open_dataset(input_data_dir_cm5 + model_ii + '/' + '1985-01_2006-01_precip.nc')
    
    regridder = xe.Regridder(ds_in, ds_out, 
                         method = 'nearest_s2d', 
                         reuse_weights = False, 
                         filename = regrid_weights_dir + 'GLDAS_ls_mask_to_' + model_ii + '.nc')
    
    ds_rgg = regridder(ds_in['GLDAS_mask'].transpose('lat','lon'))
    ds_out['land_sea_mask'] = ds_rgg
    ds_out.to_netcdf(input_data_dir_cm5 + model_ii + '/' + '1985-01_2006-01_precip_ls_mask.nc')

GFDL-ESM2M
Create weight file: /export/data1/cchristo/regridding_weights/GLDAS_ls_mask_to_GFDL-ESM2M.nc
GFDL-CM3
Create weight file: /export/data1/cchristo/regridding_weights/GLDAS_ls_mask_to_GFDL-CM3.nc
FGOALS-g2
Create weight file: /export/data1/cchristo/regridding_weights/GLDAS_ls_mask_to_FGOALS-g2.nc
IPSL-CM5A-LR
Create weight file: /export/data1/cchristo/regridding_weights/GLDAS_ls_mask_to_IPSL-CM5A-LR.nc
MIROC5
Create weight file: /export/data1/cchristo/regridding_weights/GLDAS_ls_mask_to_MIROC5.nc
MRI-ESM1
Create weight file: /export/data1/cchristo/regridding_weights/GLDAS_ls_mask_to_MRI-ESM1.nc


In [70]:
# ds_out
# ds_rgg
ds_out

<xarray.Dataset>
Dimensions:        (lat: 120, lon: 288, season: 4)
Coordinates:
  * season         (season) object 'DJF' 'JJA' 'MAM' 'SON'
  * lon            (lon) float64 0.625 1.875 3.125 4.375 ... 356.9 358.1 359.4
  * lat            (lat) float64 -59.5 -58.5 -57.5 -56.5 ... 56.5 57.5 58.5 59.5
    time           datetime64[ns] 2000-01-01
Data variables:
    mu_season      (season, lat, lon) float64 ...
    sigma_season   (season, lat, lon) float64 ...
    ampl_season    (season, lat, lon) float64 ...
    phase_season   (season, lat, lon) float64 ...
    land_sea_mask  (lat, lon) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0

## Load model grid

In [20]:
path_to_cmip_dirs = '/export/data1/cchristo/cmip6_clouds/download_from_source/'
path_to_cmip_files = path_to_cmip_dirs + 'GFDL-CM4/'
file_list = os.listdir(path_to_cmip_files)

ds_model_grid = xr.open_dataset(path_to_cmip_files + file_list[-1])

In [25]:
grid_out = xr.Dataset({'lat': ds_model_grid['lat'].values,
                       'lon': ds_model_grid['lon'].values})

## Load Satellite Grid

In [22]:
model_dir = '/export/data1/cchristo/gpm_data/gpmdata/'

result = [y for x in os.walk(model_dir) for y in glob(os.path.join(x[0], '*.nc'))]
result = sorted(result)

ds_satellite_grid = xr.open_dataset(result[10])



In [24]:
grid_in = xr.Dataset({'lat': ds_satellite_grid['lat'].values,
                       'lon': ds_satellite_grid['lon'].values})

In [15]:
ds_satellite_grid

<xarray.Dataset>
Dimensions:           (lat: 1800, lon: 3600, time: 1)
Coordinates:
  * time              (time) datetime64[ns] 2000-06-01T05:00:00
  * lon               (lon) float32 -179.95 -179.85 -179.75 ... 179.85 179.95
  * lat               (lat) float32 -89.95 -89.85 -89.75 ... 89.75 89.85 89.95
Data variables:
    precipitationCal  (lat, lon, time) float32 ...

In [13]:
grid_in

<xarray.Dataset>
Dimensions:  (lat: 1800, lon: 3600)
Coordinates:
  * lat      (lat) float32 -89.95 -89.85 -89.75 -89.65 ... 89.75 89.85 89.95
  * lon      (lon) float32 -179.95 -179.85 -179.75 ... 179.75 179.85 179.95
Data variables:
    *empty*

In [31]:
grid_out

<xarray.Dataset>
Dimensions:  (lat: 1800, lon: 3600)
Coordinates:
  * lat      (lat) float32 -89.95 -89.85 -89.75 -89.65 ... 89.75 89.85 89.95
  * lon      (lon) float32 -179.95 -179.85 -179.75 ... 179.75 179.85 179.95
Data variables:
    *empty*

In [26]:
regridder = xe.Regridder(grid_in, grid_out, 
                         method = 'bilinear', 
                         reuse_weights = False, 
                         filename = regrid_weights_dir + 'gpm_to_gfdl_cm4.nc')

Overwrite existing file: /export/data1/cchristo/regridding_weights/gpm_to_gfdl_cm4.nc 
 You can set reuse_weights=True to save computing time.
